In [ ]:
import re
import tqdm

def split_by_capitalization(transcription):
    result = []
    chunks = transcription['chunks']
    
    current_line = ""


    for chunk in chunks:
        text = chunk['text'].strip()
        start_time = chunk['timestamp'][0]
        
        # Check if the current text has capitalization at the start or in the middle of the word
        cap_search = re.search(r'[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝỲĂĐĨŨƠƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪỬỮỰỴỶỸ]', text)
        if cap_search or not current_line:
            
            if cap_search.span() != (0,1):
                split_texts = re.split(r'(?=[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝỲĂĐĨŨƠƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪỬỮỰỴỶỸ])', text)
                current_line = current_line + ' ' + ' '.join(split_texts[:-1])
                text = split_texts[-1]

            result.append(current_line.strip())
            
            # Save the current line with timestamp
            minutes = int(start_time // 60)
            seconds = start_time % 60
            timestamp = f"[{minutes:02}:{seconds:05.2f}]"
            # result.append(f"{timestamp}{current_line.strip()}")
            current_line = f"{timestamp} "+ text

        else:
            # Append to the current line
            current_line += ' ' + text
    
    
    # Add the last line if it exists
    if current_line:
        result.append(current_line.strip())

    return result


In [3]:
from transformers import pipeline
asr_pipeline = pipeline(
   "automatic-speech-recognition",
   model="xyzDivergence/whisper-medium-vietnamese-lyrics-transcription", chunk_length_s=30, device='cuda', torch_dtype='float16' ,
   return_timestamps=True,
   tokenizer="xyzDivergence/whisper-medium-vietnamese-lyrics-transcription"
)


/home/anh/miniconda3/envs/vnmese_transcription/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
transcription = asr_pipeline("sample/0 Giờ 2 Phút.mp3", return_timestamps="word")


/home/anh/miniconda3/envs/vnmese_transcription/lib/python3.9/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
WhisperModel is using WhisperSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 

In [5]:
import pprint
pprint.pprint(transcription)

{'chunks': [{'text': 'Trời', 'timestamp': (24.12, 24.68)},
            {'text': ' đêm', 'timestamp': (24.68, 25.26)},
            {'text': ' nay', 'timestamp': (25.26, 25.8)},
            {'text': ' không', 'timestamp': (25.8, 26.36)},
            {'text': ' một', 'timestamp': (26.36, 26.86)},
            {'text': ' vì', 'timestamp': (26.86, 27.36)},
            {'text': ' sao', 'timestamp': (27.36, 28.32)},
            {'text': ' Gió', 'timestamp': (28.32, 28.88)},
            {'text': ' vẫn', 'timestamp': (28.88, 29.4)},
            {'text': ' nhẹ', 'timestamp': (29.4, 29.88)},
            {'text': ' nhàng', 'timestamp': (29.88, 31.72)},
            {'text': ' Thôi', 'timestamp': (31.72, 32.3)},
            {'text': ' sao', 'timestamp': (32.3, 32.66)},
            {'text': ' lòng', 'timestamp': (32.66, 32.98)},
            {'text': ' ta', 'timestamp': (32.98, 33.32)},
            {'text': ' luôn', 'timestamp': (33.32, 33.7)},
            {'text': ' nặng', 'timestamp': (33.7, 34.46)},

In [ ]:
# Usage
formatted_text = split_by_capitalization(transcription)
print(formatted_text)